imports

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
from numpy.random import seed
import tensorflow as tf

In [ ]:
from keras.layers import Input, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import Dropout
from keras import regularizers
from keras.models import Model

Data loading

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import numpy as np


In [ ]:
columns=['Bearing 1','Bearing 2','Bearing 3','Bearing 4']
directory='dataset/'
sensor_data=pd.DataFrame()

In [ ]:
for file in os.listdir(directory):
  dataset= pd.read_csv(os.path.join(directory,file),sep='\t')
  data_mean=pd.DataFrame(np.array(dataset.abs().mean()).reshape(1,4))
  data_mean.index=[file]
  sensor_data=sensor_data.append(data_mean)

In [ ]:
sensor_data.columns=columns

In [ ]:
sensor_data_index=pd.to_datatime(sensor_data.index,format='%Y.%m.%d.%H.%M.%S.)
sensor_data=sensor_data.sort_index()
sensor_data.to_csv('Averaged_BearingTest_Dataset.csv')
print('Dataset shape:', sensor_data.shape)
sensor_data.head()


print healthy sensor data

In [ ]:
fig,ax= plt.subplots(figsize(15,8),dpi=80)
ax.plot(sensor_data[ columns[0] ], label=columns[0],color='blue',animated= True,linewidth=1)
ax.plot(sensor_data[ columns[1] ], label=columns[1],color='red',animated= True,linewidth=1)
ax.plot(sensor_data[ columns[2] ], label=columns[2],color='green',animated= True,linewidth=1)
ax.plot(sensor_data[ columns[3] ], label=columns[3],color='black',animated= True,linewidth=1)
plt.legend(loc='lower left')
ax.set_title('Bearing Train Data',fontsize=16)
plt.show()
fig.savefig('Bearing Train Data')


divide dataset to train and test set

In [ ]:
train_data=sensor_data['2004-02-12 10:52:39': '2004-02-15 12:52:39']
test_data=sensor_data['2004-02-15 12:52:39':]
print("Training dataset shape:", train_data.shape)
print("Test dataset shape:", test_data.shape)
print("Training to test ratio:", train_data.shape[0]/ sensor_data.shape[0], test_data.shape[0]/ sensor_data.shape[0])



print healthy sensor data

In [ ]:
fig,ax=plt.subplots(figsize(15,8))
ax.plot(train_data[columns[0]], label=columns[0], color='blue', animated=True,linewidth=1)
ax.plot(train_data[columns[1]], label=columns[1], color='red', animated=True,linewidth=1)
ax.plot(train_data[columns[2]], label=columns[2], color='green', animated=True,linewidth=1)
ax.plot(train_data[columns[3]], label=columns[3], color='black', animated=True,linewidth=1)
plt.legends(loc='lower_left')
ax.set_title('Bearing Train Data', fontsize= 16)
plt.show()
fig.savefig('Bearing Train Data')


transform data

In [ ]:
fft= np.fft.fft(sensor_data)
train_fft=np.fft.fft(train_data)
test_fft=np.fft.fft(test_data)

print healthy sensor frequency data

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(3,1,figsize=(20,15),dpi=80)
ax1.plot(train_fft[:0].real, label=columns[0], color='blue', animated=True, linewidth=1)
ax1.plot(train_fft[:1].real, label=columns[1], color='red', animated=True, linewidth=1)
ax1.plot(train_fft[:2].real, label=columns[2], color='green', animated=True, linewidth=1)
ax1.plot(train_fft[:3].real, label=columns[3], color='black', animated=True, linewidth=1)
plt.legend(loc='lower left')
ax1.set_title('Bearing Sensor Train Frequency Data', fontsize=16)

print degrading sensor frequency data

In [ ]:
ax2.plot(test_fft[:,0].real, label=columns[0], color='blue', animated= True, linewidth=1)
ax2.plot(test_fft[:,1].real, label=columns[1], color='red', animated= True, linewidth=1)
ax2.plot(test_fft[:,2].real, label=columns[2], color='green', animated= True, linewidth=1)
ax2.plot(test_fft[:,3].real, label=columns[3], color='black', animated= True, linewidth=1)
plt.legend(loc='lower left')
ax2.set_titles('Bearing Sensor Test Frequency Data', fontsize=16)

print frequency of whole dataset

In [ ]:
ax3.plot(fft[:,0].real, label=columns[0], color='blue', animated= True, linewidth=1)
ax3.plot(fft[:,1].real, label=columns[1], color='red', animated= True, linewidth=1)
ax3.plot(fft[:,2].real, label=columns[2], color='green', animated= True, linewidth=1)
ax3.plot(fft[:,3].real, label=columns[3], color='black', animated= True, linewidth=1)
plt.legend(loc='lower left')
ax3.set_title('Bearing Sensor All Frequency Data', fontsize=16)
plt.show()

In [ ]:
fig.savefig('Frequency of Dataset')

Preprocessing

normalize data

In [ ]:
scaler= preprocessing.MinMaxScaler()
scaled_train_data=scaler.fit_transform(train_data)
scaled_test_data=scaler.transform(test_data)

In [ ]:
scaled_train_array = scaled_train_data.reshape(scaled_train_data.shape[0], 1, scaled_train_data.shape[1])
print('Training data shape: ', scaled_train_array.shape)

Autoencoder method

create the autoencoder model

In [ ]:
inputs= Input(shape=(scaled_train_array.shape[1], scaled_train_array.shape[2]))
LSTM1 = LSTM(16, activation='relu', return_sequences= True, kernel_regularizer=regularizers.12(0.00))(inputs)
LSTM2 = LSTM(4, activation='relu', return_sequences= False)(LSTM1)
LSTM3 = RepeatVector(scaled_train_array.shape[1])(LSTM2)
LSTM4 = LSTM(4, activation='relu', return_sequences= True)(LSTM3)
LSTM5 = LSTM(16, activation='relu', return_sequences= True)(LSTM4)
out= TimeDistributed(Dense(scaled_train_array.shape[2]))(LSTM5)
model= Model(inputs=inputsi outputs=out)
model.compile(optimizer='adam', loss='mae')


fit the model to the data


In [ ]:
history = model.fit(scaled_train_array, scaled_train_array, epochs=100, batch_size=10, validation_split=0.05).history

plot the training losses

In [ ]:
fig, ax= plt. subplots(figsize=(15,8), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Autoencoder Training Loss', fontsize= 16)
ax.set_ylabel('Loss')
ax.legend(loc='upper right')
plt.show()
fig.savefig('Training history')

loss distribution for thresold estimation


In [ ]:
train_pred= model.predict(scaled_train_array)
train_pred= train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
train_pred= pd.DataFrame(train_pred, columns= columns)
train_pred.index=train_data.index

In [ ]:
scored= pd.DataFrame(index=train_data.index)
scored['Loss']= np.mean(np.abs(train_pred - scaled_train_array.reshape(scaled_train_array.shape[0], scaled_train_array.shape[2])), axis=1)
plt.figure(figsize=(16,9), dpi=80)
plt.title('Train Set Loss Distribution', fontsize=16)
sns.distplot(scored['Loss'], bins=20, kde=True, color='blue');
plt.xlim([0.0,.5])
plt.savefig('Train Set Loss Distribution')

calculate the loss on the test set

In [ ]:
test_pred= model.predict(scaled_test_array)
test_pred=test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
test_pred= pd.DataFrame(trst_pred, columns=columns)
test_pred.index= test_data.index
scored_test=pd.DataFrame(index=test_data.index)
scored_test['Loss']= np.mean(np.abs(test_pred - scaled_test_array.reshape(scaled_test_array.shape[0], scaled_test_array.shape[2])), axis= 1)
scored_test['Threshold']= 0.275
scored_test['Anomaly']= scored_test['Loss'] > scored_test['Threshold']
scored_test.head()



plot metrics for the whole dataset

In [ ]:
pred_train= model.predict(scaled_train_array)
pred_train= pred_train.reshape(pred_train.shape[0], pred_train.shape[2])
pred_train= pd.DataFrame(pred_train, columns=columns)
pred_train.index= train_data.index

scored_train['Loss']= np.mean(np.abs(pred_train - scaled_train_array.reshape(scaled_train_array[0], scaled_train_array[2])), axis=1)
scored_train['Anomaly'] =scored_train['Loss'] > scored_train['Threshold'] 
scored= pd.concat([scored_train, scored_test])


plot bearing failure time plot


In [ ]:
scored.plot(logy= True, figsize= (16,9), ylim=[1e-2,1e2], color=['blue','red'])
plt.title('Bearing Failure', fontsize= 16)
plt.savefig('Autoencoder Dataset Loss')


K-means


perform PCA

In [ ]:
pca=PCA()
pca.fit(scaled_train_data)

percentage variance

In [ ]:
per_var= np.round(pca.explained_variance_ratio_*100, decimals=1)
labels=[str(x) for x in range(1, len(per_var)+1)]
cum_var_explained= np.cumsum(per_var)

plot percentage variance

In [ ]:
fig(ax1, ax2, ax3)= plt.subplots(1, 3, figsize=(25,8))
ax1.bar(x=range(1, per_var.shape[0]+1), height= per_var, tick_label=labels)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Percentage of Explained Variable')
ax1.set_title('Screen plot', fontsize=16)



plot the pca spectrum


In [ ]:
ax2.plot(cum_var_explained, linewidth=2)
ax2.axis('tight')
ax2.set_xlabel('Number of Principal Component')
ax2.set_ylabel('Cumulative Explained Variable')
ax2.set_title('Cumulative Variance Explained', fontsize=16)


pca plot

In [ ]:
pca= PCA(n_components=2)
pca.fit(scaled_train_data)
pca_train_data= pca.transform(scaled_train_data)
pca_test_data= pca.transform(scaled_test_data)


In [ ]:
labels=['PC' + str(x) for x in range(1, pca_train_data.shape[1]+1)]
pca_train_df= pd.DataFrame(pca_train_data, columns=labels)
pca_train_df.index=train_data.index

In [ ]:
ax3.scatter(pca_train_df.PC1, pca_train_df.PC2)
ax3.set_xlabel('PC1 - {0}%'.format(per_var[0]))
ax3.set_ylabel('PC1 - {0}%'.format(per_var[1]))
ax3.set_title('Data distribution after PCA', fontsize=16)
plt.show()
fig.savefig('PCA information')

run the k means algorithm

In [ ]:
kmeans= KMeans(n_clusters=2).fit(pca_train_df)
kmeans.predict(pca_test_data)


Plot the decision boundary. For that we will assign a color to each.

In [ ]:
merged_array=sensor_data.to_numpy()

In [ ]:
h=.02
x_min, x_max= pca_train_data[:,0].min()-1, pca_train_data[:,0].max()+1
y_min, y_max= pca_train_data[:,1].min()-1, pca_train_data[:,1].max()+1
xx, yy= np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

obtain labels for each point in mesh. Use last trained model.

In [ ]:
Z= kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

put the result into a color plot

In [ ]:
Z=Z.reshape(xx.shape)
centroids= kmeans.cluster_centers_


In [ ]:
fig= plt.figure(figsize=(15,8), dpi=80)
plt.imshow(Z, interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap=plt.cm.Paired, aspect='auto', origin= 'lower')

In [ ]:
b1= plt.plot(pca_train_data[:,0], pca_train_data[:,1], 'k.', markersize=2)
b2= plt.plot(pca_train_data[:,0], pca_train_data[:,1], 'k.', markersize=2)
plt.scatter(centroids[:,0], centroids[:,1], marker='x', s=169, linewidths=3, color='w', zorder=10)
plt.title('K-means clustering')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.xlim((-1,1))
plt.ylim((-1,1))
plt.show()
fig.savefig('K-means clustering')

DBSCAN clustering method 

In [ ]:
sensor_array= sensor_data.to_numpy()

the parameter eps and min_samples were tuned experimentally

In [ ]:
dbscan= DBSCAN(eps=0.05, min_samples=10).fit(sensor_array)
core_samples_mask=np.zeros_like(dbscan.labels_,dtype=bool)
core_samples_mask[dbscan.core_sample_indices_]=True
labels=dbscan.labels_

number of clusters in labels, ignoring noise if present.

In [ ]:
n_clusters_=len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise_= list(dbscan.labels_).count(-1)
print('Estimated number of clusters: %d' %n_clusters_)
print('Estimated number of clusters: %d' %n_noise_)


In [ ]:
unique_labels=set(labels)
colors=[plt.cm.Spectral(each) for each in np.linspace(0,1,len(unique_labels))]
fig= plt.figure(1, figsize=(15,8))

In [ ]:
for k, col in zip(unique_labels,colors):
  if k==-1:
    #Black used for noise
    col=[0,0,0,1]

In [ ]:
class_member_mask= (labels==k)

In [ ]:
xy = sensor_array[class_member_mask & core_samples_mask]
plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=6)
plt.title('DBSCAN clustering')
plt.show()
fig.savefig('DBSCAN')

Isolation Forest Method

In [ ]:
iso_forest= IsolationForest(n_estimators=2)
iso_forest.fit(pca_train_data)


find if data is an outlier or not

In [ ]:
train_iso_indexes= iso_forest.predict(pca_train_data)
test_iso_indexes= iso_forest.predict(pca_test_data)

select outliers

In [ ]:
train_neg_preds=pca_train_data[train_iso_indexes==1]
test_neg_preds=pca_test_data[test_iso_indexes==1]
outliers = np.concatenate((pca_train_data[train_iso_indexes == -1], pca_test_data[test_iso_indexes == -1]), axis=0)



plot isolation forest prediction

In [ ]:
xx, yy=np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50))
Z= iso_forest.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z= Z.reshape(xx.shape)

In [ ]:
fig=plt.figure(1, figsize=(15, 8))
plt.title('Isolation Forest')
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)
b1= plt.scatter(train_neg_preds[:, 0], train_neg_preds[:, 1], c='white', s=20, edgecolor='k')
b2= plt.scatter(test_neg_preds[:, 0], test_neg_preds[:, 1], c='green', s=20, edgecolor='k')
c= plt.scatter(outliers[:, 0], outliers[:, 1], c='red', s=20, edgecolor='k')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend([b1, b2, c], ['training observations: ' +str(len(train_neg_preds)), 'new regular observations: ' +str(len(test_neg_preds)), 'new abnormal observations' +str(len(outliers))], loc='upper left')
plt.show()
fig.savefig('Isolation Forest')
print('Number of outliers:', len(outliers))



In [ ]:
def MD_detectOutliers(dist, verbose=False):
  outliers=[]
  for i in range(len(dist)):
    if dist[i]>=(np.mean(dist)*3):
      outliers.append(i)
  return np.array(outliers)

In [ ]:
def MD_threshold(dist):
  return np.mean(dist)*3

In [ ]:
def Mahalanobis_Dist(inv_cov, mean, data):
  inv_covariance_matrix=inv_cov
  temp= data-mean
  mah_dist=[]
  for i in range(len(temp)):
    mah_dist.append(np.sqrt(temp[i].dot(inv_covariance_matrix).dot(temp[i])))
  return np.array(mah_dist)

Mahalanobis Distance method

In [ ]:
pca_data_train=np.array(pca_train_df.values)
pca_data_test=np.array(pca_test_df.values)
mean= pca_data_train.mean(axis=0)
cov_matrix=np.cov(pca_data_train, rowvar=False)
inv_cov_matrix=np.linalg.inv(cov_matrix)
dist_train= Mahalanobis_Dist(inv_cov_matrix, mean, pca_data_train)
dist_test= Mahalanobis_Dist(inv_cov_matrix, mean, pca_data_test)
threshold= MD_threshold(dist_train)


In [ ]:
fig= plt.figure(figsize=(15, 8))
sns.distplot(dist_train, bins=10, kde=True, color='green');
plt.xlim([0.0,5])
plt.title('Mahalanobis dist', fontsize=16)
fig.savefig('Mahalanobis dist')

predict anomaly using Mahalanobis distance

In [ ]:
train_anomaly= pd.DataFrame()
train_anomaly['Dist']=dist_train
train_anomaly['Threshold']= threshold

If distance > threshold => anomalies

In [ ]:
anomalies['Anomaly']= anomalies['Dist'] > anomalies['Threshold']
anomalies.index=pca_test_df.index
anomalies.head()

In [ ]:
anomaly_alldata=pd.concat([train_anomaly, anomalies])
fig = anomaly_alldata.plot(logy=True, figsize=(15,8), ylim=[1e-1, 1e3], color=['green','red'], title='Mahalanobis distance for anomaly detection').get_figure()
fig.savefig('Mahalanobis distance for anomaly detection')